<a href="https://colab.research.google.com/github/GA239/DS_course/blob/master/HW7/CBtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python -V

Python 3.6.9


! cat drive/"My Drive"/asd.txt

In [3]:
# from IPython.display import Image
# try:
#   filename = take_photo()
#   print('Saved to {}'.format(filename))
  
#   # Show the image which was just taken.
#   display(Image(filename))
# except Exception as err:
#   # Errors will be thrown if the user does not have a webcam or if they do not
#   # grant the page permission to access it.
#   print(str(err))

In [4]:
! pip install tensorflow-gpu

     |████████████████████████████████| 516.2MB 30kB/s 


In [5]:
import tensorflow as tf
import os
tf.__version__

'2.2.0'

In [6]:
# from google.colab import files
# uploaded = files.upload()

In [7]:
# for k in uploaded:
#   print(k)

In [8]:
# from google.colab import files

# files.download('asd.txt')

In [9]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
with tf.device('/gpu:0'): 
  w = tf.Variable(tf.random.normal([3, 2], mean=0.0, stddev=0.4), name='weights') 

create variables

In [ ]:
w = tf.Variable(tf.random.normal([3, 2], mean=0.0, stddev=0.4), name='weights') 
b = tf.Variable(tf.zeros([2]), name='biases')

In [ ]:
w2 = tf.Variable(w.read_value(), name='w2') 

Save/Load

In [ ]:
# Add ops to save and restore all the variables.
DRIVE = '/content/drive/My Drive'
checkpoint_directory = os.path.join(DRIVE, 'training_checkpoints')
checkpoint_prefix = os.path.join(checkpoint_directory, 'ckpt')

def save(args, kwargs):
  checkpoint = tf.train.Checkpoint(**kwargs)
  checkpoint.save(file_prefix=checkpoint_prefix)

def load(args, kwargs):
  checkpoint = tf.train.Checkpoint(**kwargs)
  check_dir = tf.train.latest_checkpoint(checkpoint_directory)
  checkpoint.restore(check_dir).assert_consumed()

In [ ]:
x = tf.Variable(tf.zeros(shape=(1)), tf.float32) 
y = tf.Variable(tf.zeros(shape=(1)), tf.float32) 

@tf.function
def forward(x, y):
  return x * y


print(forward(2, 3))



tf.Tensor(6, shape=(), dtype=int32)


In [ ]:
m = tf.Variable(tf.random.normal([10, 100], mean=0.0, stddev=0.4), name='matrix') 
v = tf.Variable(tf.random.normal([100], mean=0.0, stddev=0.4), name='vector') 
m + v; 

In [ ]:
import numpy as np

n_samples, batch_size, num_steps = 1000, 100, 20000 
X_data = np.random.uniform(1, 10, (n_samples, 1)).astype(np.float32) 
y_data = 2 * X_data + 1 + np.random.normal(0, 2, (n_samples, 1)).astype(np.float32)

k = tf.Variable(tf.ones(shape=(1,1)), name="W")
b = tf.Variable(tf.zeros(shape=(1)), name="b")

display_step = 100 
optimizer = tf.optimizers.Adam()

@tf.function
def lossf(x,y):
      return tf.math.reduce_mean((y - (tf.linalg.matmul(x, k) + b)) ** 2)

def forward(x, y):
  with tf.GradientTape() as t:
      loss = lossf(x,y)
      gradients = t.gradient(loss, [k, b])
      optimizer.apply_gradients(zip(gradients, [k, b]))
  return k, b, loss

for i in range(num_steps): 
  indices = np.random.choice(n_samples, batch_size) 
  X_batch, y_batch = X_data[indices], y_data[indices] 
  k_val, b_val, lossv = forward(X_batch, y_batch)
    
  if (i+1) % display_step == 0: 
    print(f'Эпоха {i+1}:  {lossv}, k={float(k_val)}, b={float(b_val)}')


Эпоха 100:  37.97753143310547, k=1.0978363752365112, b=0.09801295399665833
Эпоха 200:  33.10771179199219, k=1.1910544633865356, b=0.19149966537952423
Эпоха 300:  27.293560028076172, k=1.2782875299453735, b=0.27926769852638245
Эпоха 400:  24.250411987304688, k=1.3607184886932373, b=0.3617456257343292
Эпоха 500:  17.163101196289062, k=1.4376417398452759, b=0.4384640157222748
Эпоха 600:  12.794303894042969, k=1.5099722146987915, b=0.5104207992553711
Эпоха 700:  13.552939414978027, k=1.5766059160232544, b=0.5767673850059509
Эпоха 800:  11.013250350952148, k=1.6366535425186157, b=0.636677622795105
Эпоха 900:  7.66900634765625, k=1.6908718347549438, b=0.6906057000160217
Эпоха 1000:  6.5172929763793945, k=1.73971688747406, b=0.7393205165863037
Эпоха 1100:  4.825112342834473, k=1.7840701341629028, b=0.7829559445381165
Эпоха 1200:  6.027284145355225, k=1.8221126794815063, b=0.8203513026237488
Эпоха 1300:  3.561466693878174, k=1.8549062013626099, b=0.8523939847946167
Эпоха 1400:  3.3250765800476

In [16]:
from keras.models import Sequential 
from keras.layers import Input, Dense, Activation 
import numpy as np

In [28]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
        
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [32]:
logr = Sequential() 
logr.add(
    Dense(1, input_dim=2, activation='sigmoid')) 
logr.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=[f1]) 

In [30]:
def sampler(n, x, y): 
  return np.random.normal(size=[n, 2]) + [x, y]

def sample_data(n=1000, p0=(-1., -1.), p1=(1., 1.)): 
  zeros, ones = np.zeros((n, 1)), np.ones((n, 1)) 
  labels = np.vstack([zeros, ones]) 
  z_sample = sampler(n, x=p0[0], y=p0[1]) 
  o_sample = sampler(n, x=p1[0], y=p1[1]) 
  return np.vstack([z_sample, o_sample]), labels

X_train, Y_train = sample_data() 
X_test, Y_test = sample_data(100) 

In [33]:
logr.fit(X_train, Y_train, batch_size=16, nb_epoch=100, verbose=1, validation_data=(X_test, Y_test))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 2000 samples, validate on 200 samples
Epoch 1/100
2000/2000 [==============================] - 0s 186us/step - loss: 0.9200 - f1: 0.1578 - val_loss: 0.8698 - val_f1: 0.0999
Epoch 2/100
2000/2000 [==============================] - 0s 152us/step - loss: 0.7835 - f1: 0.3053 - val_loss: 0.7432 - val_f1: 0.2867
Epoch 3/100
2000/2000 [==============================] - 0s 152us/step - loss: 0.6745 - f1: 0.5775 - val_loss: 0.6439 - val_f1: 0.4373
Epoch 4/100
2000/2000 [==============================] - 0s 145us/step - loss: 0.5885 - f1: 0.7736 - val_loss: 0.5657 - val_f1: 0.4889
Epoch 5/100
2000/2000 [==============================] - 0s 147us/step - loss: 0.5210 - f1: 0.8538 - val_loss: 0.5049 - val_f1: 0.5104
Epoch 6/100
2000/2000 [==============================] - 0s 150us/step - loss: 0.4681 - f1: 0.8790 - val_loss: 0.4566 - val_f1: 0.5171
Epoch 7/100
2000/2000 [==============================] - 0s 150us/step - loss: 0.4260 - f1: 0.8939 - val_loss: 0.4186 - val_f1: 0.5204
Epoch 8/